# Check MinIO

In [1]:
!pip install minio delta-spark==2.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 943.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 4.0 MB/s eta 0:00:00a 0:00:01


In [2]:
from minio import Minio

In [3]:
client = Minio(
    "minio:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False
)

bucket = "warehouse"
if client.bucket_exists(bucket):
    print(f"{bucket} exists")


warehouse exists


# Init SparkContext

In [4]:
from datetime import datetime
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext

In [5]:
spark = (SparkSession.builder.appName("pyspark-rdd-demo-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())

sqlContext = SQLContext(spark)
spark.sparkContext.getConf().getAll()


/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false'),
 ('spark.hadoop.fs.s3a.access.key', 'minio'),
 ('spark.driver.port', '43960'),
 ('spark.hadoop.fs.s3a.path.style.access', 'true'),
 ('spark.app.startTime', '1722516401004'),
 ('spark.repl.local.jars',
  'file:///usr/local/spark-3.4.0-bin-hadoop3/jars/delta-core_2.12-2.4.0.jar,file:///usr/local/spark-3.4.0-bin-hadoop3/jars/hadoop-aws-3.3.2.jar,file:///usr/local/spark-3.4.0-bin-hadoop3/jars/delta-storage-2.4.0.jar,file:///usr/local/spark-3.4.0-bin-hadoop3/jars/aws-java-sdk-1.12.507.jar,file:///usr/local/spark-3.4.0-bin-hadoop3/jars/s3-2.20.40.jar,file:///usr/local/spark-3.4.0-bin-hadoop3/jars/aws-java-sdk-bundle-1.11.1026.jar'),
 ('spark.executor.id', 'driver'),
 ('spark.jars',
  'file:///usr/local/spark-3.4.0-bin-hadoop3/jars/delta-core_2.12-2.4.0.jar,file:///usr/local/spark-3.4.0-bin-hadoop3/jars/hadoop-aws-3.3.2.jar,file:///usr/local/spark-3.4.0-bin-hadoop3/jars/delta-storage-2.4.0.jar,file:///usr/local/spark-3.4.0-bin-hadoop3/

In [6]:
sc = spark.sparkContext
sc

<SparkContext master=spark://spark-master:7077 appName=pyspark-rdd-demo-2024-08-01 12:46:39.114254>

In [7]:
# test Delta Lake
spark.range(50).write.format("delta").save("s3a://warehouse/deltafile", mode="overwrite")

# Create RDDs

## By loading dataset

In [8]:
fdd = sc.textFile("s3a://warehouse/testfile.txt")
fdd

s3a://warehouse/testfile.txt MapPartitionsRDD[29] at textFile at NativeMethodAccessorImpl.java:0

In [9]:
fdd.getNumPartitions()

2

In [10]:
fdd.count()

16

In [11]:
fdd = fdd.repartition(10)
fdd.getNumPartitions()

10

In [12]:
fdd.count()

16

In [13]:
fdd.collect()

['Contrary to popular belief, Lorem Ipsum is not simply random text. It has roots in a piece of classical Latin literature from 45 BC, making it over 2000 years old. Richard McClintock, a Latin professor at Hampden-Sydney College in Virginia, looked up one of the more obscure Latin words, consectetur, from a Lorem Ipsum passage, and going through the cites of the word in classical literature, discovered the undoubtable source. Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of "de Finibus Bonorum et Malorum" (The Extremes of Good and Evil) by Cicero, written in 45 BC. This book is a treatise on the theory of ethics, very popular during the Renaissance. The first line of Lorem Ipsum, "Lorem ipsum dolor sit amet..", comes from a line in section 1.10.32.',
 '',
 'The standard chunk of Lorem Ipsum used since the 1500s is reproduced below for those interested. Sections 1.10.32 and 1.10.33 from "de Finibus Bonorum et Malorum" by Cicero are also reproduced in their exact original form, ac

In [14]:
# split lines into words
words = fdd.flatMap(lambda line: line.split())

# count the occurrences of each word
word_counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
word_counts.collect()

[('is', 8),
 ('of', 21),
 ('45', 2),
 ('making', 3),
 ('at', 2),
 ('Hampden-Sydney', 1),
 ('looked', 1),
 ('Evil)', 1),
 ('treatise', 1),
 ('The', 4),
 ('line', 2),
 ("industry's", 1),
 ('an', 1),
 ('galley', 1),
 ('was', 1),
 ('passages,', 1),
 ('point', 1),
 ('letters,', 1),
 ("here',", 1),
 ('look', 2),
 ('default', 1),
 ('have', 2),
 ('sometimes', 2),
 ('interested.', 1),
 ('some?', 1),
 ('anything', 1),
 ('hidden', 1),
 ('necessary,', 1),
 ('true', 1),
 ('handful', 1),
 ('free', 1),
 ('standard', 2),
 ('1500s', 1),
 ('Sections', 1),
 ('and', 10),
 ('et', 2),
 ('by', 7),
 ('available,', 1),
 ('humour,', 2),
 ('which', 2),
 ("don't", 1),
 ('you', 2),
 ('predefined', 1),
 ('generator', 1),
 ('Internet.', 1),
 ('belief,', 1),
 ('simply', 2),
 ('random', 1),
 ('old.', 1),
 ('amet..",', 1),
 ('five', 1),
 ('Ipsum.', 1),
 ('normal', 1),
 ('web', 2),
 ('evolved', 1),
 ('in', 10),
 ('Latin', 4),
 ('Bonorum', 2),
 ('(The', 1),
 ('printing', 1),
 ('1500s,', 1),
 ('unknown', 1),
 ('type', 2),

In [15]:
# filter and split lines into words
words_2 = fdd.filter(lambda x: "There are" in x).flatMap(lambda line: line.split())

# count the occurrences of each word
word_counts_2 = words_2.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
word_counts_2.collect()

[('of', 6),
 ('have', 1),
 ('look', 1),
 ('anything', 1),
 ('hidden', 1),
 ('necessary,', 1),
 ('making', 1),
 ('true', 1),
 ('handful', 1),
 ('The', 1),
 ('is', 1),
 ('free', 1),
 ('available,', 1),
 ('by', 1),
 ('humour,', 2),
 ('which', 2),
 ("don't", 1),
 ('you', 2),
 ('predefined', 1),
 ('generator', 1),
 ('Internet.', 1),
 ('in', 2),
 ('injected', 2),
 ('believable.', 1),
 ('use', 1),
 ('middle', 1),
 ('repeat', 1),
 ('this', 1),
 ('Latin', 1),
 ('repetition,', 1),
 ('non-characteristic', 1),
 ('variations', 1),
 ('or', 2),
 ('slightly', 1),
 ('passage', 1),
 ('be', 1),
 ('first', 1),
 ('sentence', 1),
 ('are', 2),
 ('Ipsum', 4),
 ('alteration', 1),
 ('form,', 1),
 ('even', 1),
 ('there', 1),
 ('embarrassing', 1),
 ('generate', 1),
 ('to', 4),
 ('text.', 1),
 ('on', 2),
 ('chunks', 1),
 ('200', 1),
 ('combined', 1),
 ('structures,', 1),
 ('therefore', 1),
 ('passages', 1),
 ('but', 1),
 ('sure', 1),
 ('generators', 1),
 ('It', 1),
 ('generated', 1),
 ('etc.', 1),
 ('There', 1),
 

In [16]:
word_counts.join(word_counts_2).collect()

[('The', (4, 1)),
 ('of', (21, 6)),
 ('is', (8, 1)),
 ('have', (2, 1)),
 ('look', (2, 1)),
 ('anything', (1, 1)),
 ('hidden', (1, 1)),
 ('necessary,', (1, 1)),
 ('making', (3, 1)),
 ('true', (1, 1)),
 ('handful', (1, 1)),
 ('free', (1, 1)),
 ('by', (7, 1)),
 ('available,', (1, 1)),
 ('humour,', (2, 2)),
 ('which', (2, 2)),
 ("don't", (1, 1)),
 ('you', (2, 2)),
 ('predefined', (1, 1)),
 ('generator', (1, 1)),
 ('Internet.', (1, 1)),
 ('in', (10, 2)),
 ('injected', (2, 2)),
 ('believable.', (1, 1)),
 ('use', (3, 1)),
 ('middle', (1, 1)),
 ('repeat', (1, 1)),
 ('this', (1, 1)),
 ('Latin', (4, 1)),
 ('repetition,', (1, 1)),
 ('non-characteristic', (1, 1)),
 ('variations', (1, 1)),
 ('or', (2, 2)),
 ('slightly', (1, 1)),
 ('passage', (1, 1)),
 ('be', (2, 1)),
 ('first', (2, 1)),
 ('sentence', (1, 1)),
 ('Ipsum', (13, 4)),
 ('are', (3, 2)),
 ('form,', (2, 1)),
 ('alteration', (1, 1)),
 ('even', (1, 1)),
 ('there', (1, 1)),
 ('embarrassing', (1, 1)),
 ('generate', (1, 1)),
 ('to', (7, 4)),
 (

In [17]:
# create two RDDs with key-value pairs
rdd1 = sc.parallelize([(1, "hello"), (2, "world"), (3, "foo")], 1)
rdd2 = sc.parallelize([(1, "bar"), (2, "baz"), (4, "qux")], 1)

# perform a join on the two RDDs
rdd_joined = rdd1.join(rdd2)

# print the result
rdd_joined.collect()

[(2, ('world', 'baz')), (1, ('hello', 'bar'))]

## By using parallelize

In [18]:
data = [1, 2, 3, 3]
rdd = sc.parallelize(data, 2)
rdd

ParallelCollectionRDD[61] at readRDDFromFile at PythonRDD.scala:287

In [19]:
rdd.getNumPartitions()

2

In [20]:
rdd.collect()

[1, 2, 3, 3]

# RDD operations

## Transformation

* Element-wise transformations
* Transformation filter(): Takes in a function and returns an RDD that only has elements that pass the filter( ) function

In [21]:
rdd.filter(lambda x: x != 1).collect()

[2, 3, 3]

* Element-wise transformations
* Transformation map(): Takes in a function and applies it to each element  in the RDD with the result of the function being the new value of each element in the resulting RDD

In [22]:
rdd.map(lambda x: x + 1).collect()

[2, 3, 4, 4]

In [23]:
rdd.map(lambda x: [x, x + 5]).collect()

[[1, 6], [2, 7], [3, 8], [3, 8]]

In [24]:
rdd.flatMap(lambda x: [x, x + 5]).collect()

[1, 6, 2, 7, 3, 8, 3, 8]

* Sampling Transformation
* sample() an RDD: We can specify with or without replacement, or the fraction

In [25]:
rdd.sample(False, 0.5).collect()

[]

* Pseudo Set Operations
* RDDs support many operations of mathematical sets: 
* distinct, union, intersect, subtract
* All expensive except union because they involve shuffling

In [26]:
rdd = sc.parallelize([1, 2, 3])
other = sc.parallelize([3, 4, 5])

In [27]:
rdd.union(other).collect()

[1, 2, 3, 3, 4, 5]

In [28]:
rdd.intersection(other).collect()

[3]

In [29]:
rdd.subtract(other).collect()

[1, 2]

In [30]:
rdd.cartesian(other).collect()

[(1, 3), (1, 4), (1, 5), (2, 3), (3, 3), (2, 4), (2, 5), (3, 4), (3, 5)]

## Actions

* fold( )
* takes a function similarly as reduce( ) does, but takes a “zero value” to be used for initial call on each partition
* should be the identity element for the operation
* 0 for +, 1 for *, etc.
* return type the same as RDD elements 


In [31]:
rdd = sc.parallelize([1, 2, 3, 3])
rdd.fold(0, lambda a, b: a + b)

9

* reduce( )
* takes a function that operates on two elements of the type in the RDD and returns a new element of the same type
* should be commutative and associative so that it can be computed correctly in parallel

In [32]:
rdd.reduce(lambda a, b: a + b)

9

aggregate( )
* we can also supply an initial zero value of the type we want to return
* a 1st function to combine the elements from RDD with the accumulator
* a 2nd function to merge two accumulators given that each node accumulates its own results locally

In [33]:
seq_op = (lambda acc, value: (acc[0] + value, acc[1] + 1))
comb_op = (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]))

sumCount = rdd.aggregate((0, 0), seq_op, comb_op)
sumCount[0], sumCount[1], sumCount[0]/float(sumCount[1])

(9, 4, 2.25)

collect( )
* return the entire RDD’s contents to the driver program

take( )
* returns n elements from the RDD
* attempts to minimize the number of partitions it accesses, so may be biased

top(n)
* return the top n elements of the RDD

count( )
* returns the number of elements in the RDD

In [34]:
rdd.collect()

[1, 2, 3, 3]

In [35]:
rdd.take(2)

[1, 2]

In [36]:
rdd.takeOrdered(2, key=lambda x: -x)

[3, 3]

In [37]:
rdd.takeSample(False, 1)

[2]

In [38]:
rdd.top(2)

[3, 3]

In [39]:
rdd.count()

4

In [40]:
rdd.countByValue()

defaultdict(int, {1: 1, 2: 1, 3: 2})

# Caching RDDs

In [41]:
lines = sc.textFile("s3a://warehouse/testfile.txt", 4)
lines

s3a://warehouse/testfile.txt MapPartitionsRDD[99] at textFile at NativeMethodAccessorImpl.java:0

Count will cause Spark to
* read data
* sum within partitions
* combine sums in driver

In [42]:
lines.count()

16

In [43]:
paragraphs = lines.filter(lambda x: len(x) > 0)
paragraphs

PythonRDD[101] at RDD at PythonRDD.scala:53

Count will cause Spark to
* read data (again)
* sum within partitions
* combine sums in driver

In [44]:
paragraphs.count()

9

In [45]:
lines = sc.textFile("s3a://warehouse/testfile.txt", 4)

# save, don't compute
lines.cache()

s3a://warehouse/testfile.txt MapPartitionsRDD[104] at textFile at NativeMethodAccessorImpl.java:0

In [46]:
paragraphs = lines.filter(lambda x: len(x) > 0)
print(lines.count())

16


In [47]:
print(paragraphs.count())

9


In [48]:
# remove from cache
lines.unpersist()

s3a://warehouse/testfile.txt MapPartitionsRDD[104] at textFile at NativeMethodAccessorImpl.java:0

In [49]:
sc.stop()